Calculate all combinations of target distances  
This is used for ML as label data

In [1]:
import os
import sys
import numpy as np
import xarray as xr

module_path = os.path.abspath(os.path.join('../src'))
if module_path not in sys.path:
    sys.path.append(module_path)

from utils import to_monthly

In [2]:
# Parameters
vname = 'sst'
grid = '2x2'
lat_slice = (-10, 10)
lon_slice = (120, 290)
data_dir = '../data/cesm2'

# Read data
f = f'{data_dir}/{vname}_anomaly_{grid}.nc'
da = xr.open_dataarray(f)
da = da.sel(lat=slice(*lat_slice), lon=slice(*lon_slice))
da_month = to_monthly(da)

In [3]:
# Rename
da_library = da_month.rename({'ens': 'lens', 'year': 'lyear'})

# Grid weights
wgt = np.cos(np.deg2rad(da.lat))

In [4]:
%%time
msd_enss = []
for ens in da_month.ens.data:
    print(ens)
    msd_years = []
    for year in da_month.year.data:
        msd = ((da_month.sel(ens=ens, year=year)
                - da_library) ** 2
                ).weighted(wgt).mean(dim=['lat', 'lon'])
        msd_years.append(msd)
    msd_enss.append(xr.concat(msd_years, dim='year'))
msd = xr.concat(msd_enss, dim='ens')

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40


In [ ]:
# Save
encoding = {vname: {'dtype': 'float32'}}
outf = f'{data_dir}/target_distance.nc'
msd.to_netcdf(outf, encoding=encoding)